In [ ]:
pip install numpy

## Importing necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.metrics import F1Score

import cv2
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd

from glob import glob
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [ ]:
img_width = 256
img_height = 256

images_train = []

# print(mask_files)
data_path = '/kaggle/input/lgg-mri-segmentation/kaggle_3m'

In [ ]:
mask_files = glob(pathname= data_path+'/*/*_mask*')
mask_files[:1]

In [ ]:
for i in mask_files:
  images_train.append(i.replace('_mask', ''))
print(images_train[:5])


In [ ]:
len(images_train)

## Data Visualization


Here are the few samples of the data



In [ ]:
def plot_from_path(rows, columns, img_list_path, mask_list_path):
  fig = plt.figure( figsize = (12,12))
  for i in range(1, rows*columns+1):
    fig.add_subplot(rows,columns, i)
    img_path = img_list_path[i]
    mask_path = mask_list_path[i]
    image = cv2.imread(img_path)
    image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path)
    plt.imshow(image)
    plt.imshow(mask, alpha = 0.05)
  plt.show()

In [ ]:
plot_from_path(5,5, images_train, mask_files)

## Data Preprocessing

### Splitting into test, train and validation


We split the data into train, test and validation datas alloting them 85%, 10% and 5% datas respectively

In [ ]:
df = pd.DataFrame( data = {
    'images_train': images_train,
    'masks': mask_files
})
df_train, df_test = train_test_split(df, test_size = 0.10)
df_train, df_valid = train_test_split(df_train, test_size=0.05)

print(df_train.shape)
print(df_test.shape)
print(df_valid.shape)

### Generators, Augmentation, Normalization and Loss Calculator

In [ ]:
def train_generator(
    data_frame,
    batch_size,
    augmentation_dict,
    image_color_mode="rgb",
    mask_color_mode="grayscale",
    image_save_prefix="image",
    mask_save_prefix="mask",
    save_to_dir=None,
    target_size=(256, 256),
    seed=1,
):

    image_datagen = ImageDataGenerator(**augmentation_dict)
    mask_datagen = ImageDataGenerator(**augmentation_dict)

    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col="images_train",
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed,
    )

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col="masks",
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed,
    )

    train_gen = zip(image_generator, mask_generator)


    for (img, mask) in train_gen:
        img, mask = normalize_data(img, mask)
        yield (img, mask)

In [ ]:
def normalize_data(img,mask):
    img = img / 255
    mask = mask / 255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0

    return (img, mask)

In [ ]:
def dice_coefficients(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coefficients_loss(y_true, y_pred, smooth=100):
    return -dice_coefficients(y_true, y_pred, smooth)
def iou(y_true, y_pred, smooth=100):
    intersection = K.sum(y_true * y_pred)
    sum = K.sum(y_true + y_pred)
    iou = (intersection + smooth) / (sum - intersection + smooth)
    return iou
def jaccard_distance(y_true, y_pred):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    return -iou(y_true_flatten, y_pred_flatten)

## Defining encoder and decoder block

In [ ]:
def encoder_block(inputs, filters):
    conv = Conv2D(filters, kernel_size = (3,3), padding="same")(inputs)
    bn = Activation("relu")(conv)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same")(bn)
    bn = BatchNormalization(axis=3)(conv)
    bn = Activation("relu")(bn)
    pool = MaxPooling2D(pool_size=(2, 2))(bn)

    return conv, pool;

def decoder_block(inputs, conv, num_filters):
  conv_trans = Conv2DTranspose(num_filters, kernel_size= (2,2), strides = (2,2), padding = "same")(inputs)

  x = concatenate([conv_trans, conv], axis = 3)
  x = Conv2D(num_filters, kernel_size = (3,3), padding= "same")(x)
  x = Activation("relu")(x)
  x = Conv2D(num_filters, kernel_size = (3,3), padding = "same")(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)

  return x ;





## Defining the U-Net Architecture

In [ ]:
def unet_model(input_shape = (img_width, img_height, 3)):
  inputs = Input(input_shape)

  #Contracting paths ( Encoder Path) / Downsampling

  conv1, pool1 = encoder_block( inputs, 64)
  conv2, pool2 = encoder_block(pool1, 128)
  conv3, pool3 = encoder_block(pool2, 256)
  conv4, pool4 = encoder_block(pool3, 512)


  # Bottleneck
  b1 = Conv2D(filters = 1024, kernel_size = (3,3), padding = 'same')(pool4)
  b1 = Activation('relu')(b1)
  b1 = Conv2D(filters = 1024, kernel_size = (3,3), padding = 'same')(b1)
  b1 = BatchNormalization(axis = 3)(b1)
  b1 = Activation('relu')(b1)

  #Expansive path ( Decoder path) / UpSampling

  s5 = decoder_block(b1, conv4, 512)
  s6 = decoder_block(s5, conv3, 256)
  s7 = decoder_block(s6, conv2, 128)
  s8 = decoder_block(s7, conv1 , 64)


  #Output
  outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(s8)

  return Model(inputs=[inputs], outputs = [outputs])

## training parameters and augmentation dictionary

In [ ]:
epochs = 5
batch_size = 32
lr = 1e-4

train_generator_args = dict(
    rotation_range = 0.25,
    width_shift_range = 0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen  = train_generator( df_train, batch_size, train_generator_args)
test_gen = train_generator(df_test, batch_size, dict() )


model = unet_model(input_shape =(256,256, 3))
optimizer = Adam(lr = lr , beta_1 = 0.9, beta_2 = 0.999, epsilon = None, amsgrad = False)


model.compile(optimizer=optimizer, loss=dice_coefficients_loss, metrics=["binary_accuracy", iou, dice_coefficients])
callbacks = [
    ModelCheckpoint('unet_model.hdf5',verbose=1, save_best_only=True )
]

history = model.fit(
      train_gen,
      steps_per_epoch = len(df_train) / batch_size,
      epochs = epochs,
      callbacks = callbacks,
      validation_data = test_gen,
      validation_steps = len(df_valid) / batch_size
)

In [ ]:
keras.utils.plot_model(model, to_file = "Model.png", show_shapes=True)


In [ ]:
history_training = history.history

train_dice_coeff_list = history_training['dice_coefficients']
test_dice_coeff_list = history_training['val_dice_coefficients']

train_jaccard_list = history_training['iou']
test_jaccard_list = history_training['val_iou']

train_loss_list = history_training['loss']
test_loss_list = history_training['val_loss']


plt.plot(test_loss_list, 'b-', label='Test Loss')
plt.plot(train_loss_list, 'r-', label='Train Loss')

plt.xlabel('iterations')
plt.ylabel('loss')
plt.title('loss graph', fontsize=12)
plt.legend()
plt.show()
plt.savefig('Loss Graph')

In [ ]:
plt.plot(train_dice_coeff_list, 'b-', label='Train Accuracy')
plt.plot(test_dice_coeff_list, 'r-', label = 'Test Accuracy')

plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.title('Accuracy graph', fontsize=12)
plt.legend()
plt.show()
plt.savefig('Accuracy Graph')

In [ ]:
model = load_model('unet_model.hdf5', custom_objects={'dice_coefficients_loss': dice_coefficients_loss, 'iou': iou, 'dice_coefficients': dice_coefficients})

In [ ]:
test_gen = train_generator(df_test, batch_size, dict(), target_size=(img_height,img_width))

results = model.evaluate(test_gen, steps = len(df_test)/batch_size)
print('Test Loss', results[0])
print('Test IOU', results[1])
print('Test Dice Coeff', results[2])

In [ ]:
import numpy as np
for i in range(20):
    index = np.random.randint(1, len(df_test.index))
    img = cv2.imread(df_test['images_train'].iloc[index])
    img = cv2.resize(img, (img_height,img_width))
    img = img/255
    img = img[np.newaxis, : ,:, :]
    pred_img = model.predict(img)
    
    plt.figure(figsize=(12,12))
    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(img))
    plt.title("Original Image")
    plt.subplot(1, 3, 2)
    plt.imshow(np.squeeze(cv2.imread(df_test['masks'].iloc[index])))
    plt.title("Original Mask")
    plt.subplot(1, 3, 3)
    plt.imshow(np.squeeze(pred_img) > 0.5)
    plt.title("Prediction")
    plt.show()
    plt.savefig('Originals vs Predictions')

In [ ]:
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
import seaborn as sns
cm = confusion_matrix(y_true.flatten(), y_pred.flatten())
